In [1]:
!apt-get install -y cuda-toolkit-12-3

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  ca-certificates-java cuda-cccl-12-3 cuda-command-line-tools-12-3
  cuda-compiler-12-3 cuda-crt-12-3 cuda-cudart-12-3 cuda-cudart-dev-12-3
  cuda-cuobjdump-12-3 cuda-cupti-12-3 cuda-cupti-dev-12-3 cuda-cuxxfilt-12-3
  cuda-documentation-12-3 cuda-driver-dev-12-3 cuda-gdb-12-3
  cuda-libraries-12-3 cuda-libraries-dev-12-3 cuda-nsight-12-3
  cuda-nsight-compute-12-3 cuda-nsight-systems-12-3 cuda-nvcc-12-3
  cuda-nvdisasm-12-3 cuda-nvml-dev-12-3 cuda-nvprof-12-3 cuda-nvprune-12-3
  cuda-nvrtc-12-3 cuda-nvrtc-dev-12-3 cuda-nvtx-12-3 cuda-nvvm-12-3
  cuda-nvvp-12-3 cuda-opencl-12-3 cuda-opencl-dev-12-3 cuda-profiler-api-12-3
  cuda-sanitizer-12-3 cuda-toolkit-12-3-config-common cuda-tools-12-3
  cuda-visual-tools-12-3 default-jre default-jre-headless fonts-dejavu-core
  fonts-dejavu-extra gds-tools-12-3 java-common libatk-wrapper-java
  libat

In [2]:
!pip install nvcc4jupyter

In [54]:
!python --version
!nvcc --version
%load_ext nvcc4jupyter

!rm -f /usr/local/cuda
!ln -s /usr/local/cuda-12.3 /usr/local/cuda

%reload_ext nvcc4jupyter



Python 3.12.12
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Wed_Nov_22_10:17:15_PST_2023
Cuda compilation tools, release 12.3, V12.3.107
Build cuda_12.3.r12.3/compiler.33567101_0
The nvcc4jupyter extension is already loaded. To reload it, use:
  %reload_ext nvcc4jupyter
Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpk4lr552p".


In [148]:
%%cuda -arch=compute_75
// serial cpp code
#include <cuda_runtime.h>
#include <stdio.h>
#include <random>
#include <iostream>
#include <chrono>
#include <iomanip>

int simulation(int* circle_points, int* square_points, int N, std::mt19937& gen, std::uniform_real_distribution<float>& dis) {
    float x, y;
    for (int n = 0; n < N; ++n) {
        x = dis(gen);
        y = dis(gen);
        if (x*x + y*y <= 1)
            (*circle_points)++;
        else
          (*square_points)++;
    }
}

int main() {
    // random number generator
    //std::random_device rd;  // Will be used to obtain a seed for the random number engine
    int seed = 12; // set seed
    std::mt19937 gen(seed); // Standard mersenne_twister_engine seeded with rd()
    std::uniform_real_distribution<float> dis(0.0, 1.0);

    // declare values
    int N = 1e6;  // number of simulations
    int circle_points;
    int square_points;
    circle_points = 0;
    square_points = 0;
    const auto start = std::chrono::high_resolution_clock::now();

    simulation(&circle_points, &square_points, N, gen, dis);

    const auto end = std::chrono::high_resolution_clock::now();

    std::chrono::duration<double, std::milli> elapsed_ms = end - start;

    float pi = 4.0f * static_cast<float>(circle_points) / (circle_points + square_points);

    std::cout << std::setprecision(std::numeric_limits<double>::max_digits10);

    std::cout << "pi = " << pi << " | Simulations = " << N << " | elapsed time(ms) " << elapsed_ms.count() << std::endl;
    return 0;
}


pi = 3.14357590675354 | Simulations = 1000000 | elapsed time(ms) 96.205254999999994



In [145]:
%%writefile monte_carlo.cu

#include <cuda_runtime.h>
#include <stdio.h>
#include <random>
#include <iostream>
#include <curand.h>

__global__
void compute(int* circle_points, int* square_points, size_t N, float* xVec, float* yVec, int unrolls) {
    __shared__ int s_circle;
    __shared__ int s_square; // use shared memory for less overhead
    if (threadIdx.x == 0) {
        s_circle = 0;
        s_square = 0;
    }
    __syncthreads();

    size_t idx = blockIdx.x * blockDim.x * unrolls + threadIdx.x;
    size_t i;
    int localC = 0;
    int localS = 0;

    #pragma unroll 16
    for (int n = 0; n < unrolls; n++) {
        i = idx + n * blockDim.x;
        if (i < N) {
            float x = xVec[i];
            float y = yVec[i];
            if (x*x + y*y <= 1)
                localC++;
            else
                localS++;
        }
    }
    atomicAdd(&s_circle, localC); // added to shared memory
    atomicAdd(&s_square, localS);
    __syncthreads();

    if (threadIdx.x == 0) {
        atomicAdd(circle_points, s_circle);
        atomicAdd(square_points, s_square);
    }
}

void simulation(int* circle_points, int* square_points, size_t N, curandGenerator_t& generator, float* xVec, float* yVec) {
    // standard real mersenne float monte carlo distribution
    curandGenerateUniform(generator, xVec, N);
    curandGenerateUniform(generator, yVec, N);
    int threads = 256;
    int unrolls = 8;
    int blocks = (N + threads * unrolls -1)/(threads*unrolls);

    compute<<<blocks, threads>>>(circle_points, square_points, N, xVec, yVec, unrolls);
    cudaDeviceSynchronize(); // ensure GPU finishes before reading counters

}

float benchMark(int* circle_points, int* square_points, size_t N, curandGenerator_t& gen, float* xVec, float* yVec, bool flush) {
    // cache flushing
    int device = 0;
    int l2_size = 0;
    float* d_F;

    cudaGetDevice(&device);
    cudaDeviceGetAttribute(&l2_size, cudaDevAttrL2CacheSize, device);
    size_t sizeF = l2_size * 2;

    // benchmarking variables
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    if (flush){
        cudaMalloc((void **)&d_F, sizeF);
        cudaMemsetAsync((void *)d_F, 0, sizeF);
        cudaDeviceSynchronize();
    }

    cudaEventRecord(start, 0);
    simulation(circle_points, square_points, N, gen, xVec, yVec);

    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);

    if (flush){
        cudaFree(d_F);
    }

    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    return milliseconds;
}

int main() {
    size_t N = 10000000;

    // Use regular device memory for counters instead of Unified Memory
    int *d_circle_points, *d_square_points;
    int h_circle_points = 0, h_square_points = 0;

    float *xVec, *yVec;
    float milliseconds;

    // Allocate device memory for counters
    cudaMalloc(&d_circle_points, sizeof(int));
    cudaMalloc(&d_square_points, sizeof(int));
    cudaMalloc(&xVec, N * sizeof(float));
    cudaMalloc(&yVec, N * sizeof(float));

    // Initialize counters on device
    cudaMemset(d_circle_points, 0, sizeof(int));
    cudaMemset(d_square_points, 0, sizeof(int));

    // Random number generator setup
    unsigned long long seed = 12;
    curandGenerator_t generator;
    curandCreateGenerator(&generator, CURAND_RNG_PSEUDO_MT19937);
    curandSetPseudoRandomGeneratorSeed(generator, seed);

    milliseconds = benchMark(d_circle_points, d_square_points, N, generator, xVec, yVec, false);

    // Copy results back from device to host
    cudaMemcpy(&h_circle_points, d_circle_points, sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(&h_square_points, d_square_points, sizeof(int), cudaMemcpyDeviceToHost);

    float pi = 4.0f * static_cast<float>(h_circle_points) / (h_circle_points + h_square_points);

    std::cout << "pi = " << pi << " | Simulations = " << N << " | elapsed time(ms) " << milliseconds << std::endl;

    cudaFree(d_circle_points);
    cudaFree(d_square_points);
    cudaFree(xVec);
    cudaFree(yVec);
    curandDestroyGenerator(generator);

    return 0;
}


Overwriting monte_carlo.cu


In [146]:
!nvcc -arch=compute_75 -code=sm_75 monte_carlo.cu -o monte_carlo -lcurand

In [147]:
!./monte_carlo

pi = 3.14145 | Simulations = 10000000 | elapsed time(ms) 310.127
